**Check GPU status**

In [1]:
!nvidia-smi

Tue Mar 11 09:22:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Implementation of DeepSeek Finetuning using Unsloth and Ollama
Inspired from https://unsloth.ai/blog/deepseek-r1

**Install packages**

In [2]:
!pip3 install torch
!pip3 install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
!pip install unsloth

!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 5.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.6/191.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1

Found existing installation: unsloth 2025.3.9
Uninstalling unsloth-2025.3.9:
  Successfully uninstalled unsloth-2025.3.9
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-e_6sctfq
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-e_6sctfq
  Resolved https://github.com/unslothai/unsloth.git to commit 2b5d81d75281c02480927cf3ca0dea7c8e98d484
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.3.9-py3-none-any.whl size=191199 sha256=c109f5d39d95948f0aeab19882b548905842e2cf3ccfeddfbbfa259f3c457b27
  Stored in directory: /tmp/pip-ephem-wheel-cache-m4dlq60q/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth


**Load the pretrained model : DeepSeek-R1-Distill**

In [4]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.9: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]


*   Configures the PEFT (Parameter-Efficient Fine-Tuning) model using FastLanguageModel.

* This method applies LoRA (Low-Rank Adaptation) to the specified attention projection modules (q_proj, k_proj, v_proj, o_proj), sets hyperparameters for LoRA (e.g., lora_alpha, lora_dropout), and adjusts model behavior (e.g., gradient checkpointing, bias handling).

* The 'unsloth' setting for gradient checkpointing is specified, and RSLORA is disabled.

* The random seed is set to 42 for reproducibility, and loftq_config is set to None, implying no custom configuration for loftq.


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 4,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.3.9 patched 32 layers with 32 QKV layers, 32 O layers and 0 MLP layers.


**Data preparation**

In [6]:
from datasets import load_dataset
dataset = load_dataset("Josephgflowers/Finance-Instruct-500k", split = "train")
print(dataset.column_names)

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

train.json:   0%|          | 0.00/580M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/518185 [00:00<?, ? examples/s]

['system', 'user', 'assistant']


In [7]:
dataset[0]

{'system': '\n',
 'user': "Explain tradeoffs between fiscal and monetary policy as tools in a nation's economic toolkit. Provide examples of past instances when each were utilized, the economic conditions that led to them being deployed, their intended effects, and an evaluation of their relative efficacy and consequences.",
 'assistant': 'Fiscal and monetary policy are the two main tools that governments have to influence economic activity. They each have benefits and drawbacks.\n\nFiscal policy refers to government spending and taxation decisions. Examples of fiscal policy include:\n\n• During the Great Recession, the U.S. government implemented a fiscal stimulus through the American Recovery and Reinvestment Act of 2009. This included increased spending on infrastructure, tax cuts, and expanded unemployment benefits. The intention was to boost aggregate demand and stimulate economic activity. Studies have found that the stimulus had a positive but modest impact on economic growth.\n

**Converts the dataset into a ShareGPT-compatible format using the `to_sharegpt` function from the unsloth library**


In [8]:
from unsloth import to_sharegpt

dataset = to_sharegpt(
    dataset,
    merged_prompt = "{user}[[\nYour input is:\n{system}]]", # Changed merged_prompt to use available columns
    output_column_name = "assistant", # Changed output_column_name to use available columns
    conversation_extension = 3,
)

Merging columns:   0%|          | 0/518185 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/518185 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/518185 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/518185 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/518185 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/518185 [00:00<?, ? examples/s]

In [10]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

Standardizing format:   0%|          | 0/518185 [00:00<?, ? examples/s]

In [11]:
dataset[0]['conversations']

[{'content': "Explain tradeoffs between fiscal and monetary policy as tools in a nation's economic toolkit. Provide examples of past instances when each were utilized, the economic conditions that led to them being deployed, their intended effects, and an evaluation of their relative efficacy and consequences.\nYour input is:\n\n",
  'role': 'user'},
 {'content': 'Fiscal and monetary policy are the two main tools that governments have to influence economic activity. They each have benefits and drawbacks.\n\nFiscal policy refers to government spending and taxation decisions. Examples of fiscal policy include:\n\n• During the Great Recession, the U.S. government implemented a fiscal stimulus through the American Recovery and Reinvestment Act of 2009. This included increased spending on infrastructure, tax cuts, and expanded unemployment benefits. The intention was to boost aggregate demand and stimulate economic activity. Studies have found that the stimulus had a positive but modest imp

**Customizable Chat Templates**

In [12]:
chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/518185 [00:00<?, ? examples/s]

**Train a model**

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 40,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


Tokenizing to ["text"] (num_proc=2):   0%|          | 0/518185 [00:00<?, ? examples/s]

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 518,185 | Num Epochs = 1 | Total steps = 40
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 3,407,872/4,632,088,576 (0.07% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.480800
2,2.121400
3,2.350300
4,2.407500
5,2.311100
6,1.815000
7,2.077600
8,1.985900
9,2.025000
10,2.005600


**Ollama**

In [15]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [16]:
model.save_pretrained_gguf("model", tokenizer)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.4 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 53%|█████▎    | 17/32 [00:00<00:00, 30.86it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [02:19<00:00,  4.37s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: /content/model/unsloth.Q8_0.gguf
Unsloth: Saved Ollama Modelfile to model/Modelfile


In [17]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3)

In [18]:
print(tokenizer._ollama_modelfile)

FROM {__FILE_LOCATION__}

TEMPLATE """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.{{ if .Prompt }}

### Instruction:
{{ .Prompt }}{{ end }}

### Response:
{{ .Response }}<｜end▁of▁sentence｜>"""

PARAMETER stop "<think>"
PARAMETER stop "<|finetune_right_pad_id|>"
PARAMETER stop "<｜▁pad▁｜>"
PARAMETER stop "</think>"
PARAMETER stop "<|python_tag|>"
PARAMETER stop "<｜Assistant｜>"
PARAMETER stop "<|eot_id|>"
PARAMETER stop "<|end_header_id|>"
PARAMETER stop "<|start_header_id|>"
PARAMETER stop "<|eom_id|>"
PARAMETER stop "<｜User｜>"
PARAMETER stop "<｜end▁of▁sentence｜>"
PARAMETER stop "<|reserved_special_token_"
PARAMETER temperature 1.5
PARAMETER min_p 0.1


In [19]:
!ollama create deepseek_finetuned_finance_model_TM -f ./model/Modelfile

gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠴ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠹ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠏ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering mode

In [20]:
!pip3 install ollama

In [21]:
import ollama

response = ollama.chat(model="deepseek_finetuned_finance_model_TM",
            messages=[{ "role": "user", "content": "What is the difference between home loan and consumer loan? Explain with example."
            },
                      ])

print(response.message.content)

Home loans are specifically for purchasing or renovating a property you will use as your primary residence, whereas consumer loans can be used for any personal purpose. For instance, if you're looking to buy a house, take out $200,000 as a home loan; but if you need to cover unexpected medical costs, take a $10,000 consumer loan.


In [23]:
response = ollama.chat(model="deepseek_finetuned_finance_model_TM",
            messages=[{ "role": "user", "content": "Explain me budgeting strategy for an IT employee in detail."
            },
                      ])

print(response.message.content)

Budgeting strategies can vary based on your personal financial goals and priorities, but here’s a general approach you could take: 1. **Identify Your Income Sources:** Start by determining all the sources of income that come into your life regularly. These include salaries, stipends, bonuses, commission payments, etc. 2. **Determine Your Financial Goals:** Set clear financial goals for each of your spending categories (e.g., housing costs, travel expenses, food, entertainment, savings) so you can allocate your resources effectively. 3. **Break Down Expenses:** Create a detailed list of all the expenses that are part of your budget. Group similar expenses together to make the process more manageable (e.g., utilities under one category). 4. **Create an Income and Expense Spreadsheet:** This will give you a clearer picture of where your money is going each month. Input your income, subtract your expenses, and keep track of savings or surplus in this document. 5. **Monitor and Adjust Your 